<a href="https://colab.research.google.com/github/BachokunIAR/Battery/blob/main/Li_materials_battery.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#<font color=red>1. Importing functions

In [8]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import sklearn
from sklearn.model_selection import train_test_split

In [9]:
from sklearn.metrics import mean_squared_error as mean_squared_error
from sklearn.metrics import r2_score
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.impute import SimpleImputer
try:
  from jarvis.db.figshare import data
except:
  !pip install -q jarvis-tools
  from jarvis.db.figshare import data


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 22.2 MB/s eta 0:00:00


In [10]:
from jarvis.db.figshare import data

# Load JARVIS-DFT dataset (~40k materials)
dft = data("dft_3d")

print(len(dft), dft[0]["jid"], dft[0]["density"])


Obtaining 3D dataset 76k ...
Reference:https://www.nature.com/articles/s41524-020-00440-1
Other versions:https://doi.org/10.6084/m9.figshare.6815699


100%|██████████| 40.8M/40.8M [00:02<00:00, 18.5MiB/s]


Loading the zipfile...
Loading completed.
75993 JVASP-90856 5.956


In [11]:
import pandas as pd
df = pd.DataFrame(dft)
df = df.head()
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
df

,jid,spg_number,spg_symbol,formula,formation_energy_peratom,func,optb88vdw_bandgap,atoms,slme,magmom_oszicar,spillage,elastic_tensor,effective_masses_300K,kpoint_length_unit,maxdiff_mesh,maxdiff_bz,encut,optb88vdw_total_energy,epsx,epsy,epsz,mepsx,mepsy,mepsz,modes,magmom_outcar,max_efg,avg_elec_mass,avg_hole_mass,icsd,dfpt_piezo_max_eij,dfpt_piezo_max_dij,dfpt_piezo_max_dielectric,dfpt_piezo_max_dielectric_electronic,dfpt_piezo_max_dielectric_ionic,max_ir_mode,min_ir_mode,n-Seebeck,p-Seebeck,n-powerfact,p-powerfact,ncond,pcond,nkappa,pkappa,ehull,Tc_supercon,dimensionality,efg,xml_data_link,typ,exfoliation_energy,spg,crys,density,poisson,raw_files,nat,bulk_modulus_kv,shear_modulus_gv,mbj_bandgap,hse_gap,reference,search
0,JVASP-90856,129,P4/nmm,TiCuSiAs,-0.42762,OptB88vdW,0.000,"{'lattice_mat': [[3.566933224304235, 0.0, -0.0...",na,0.0,na,na,"{'p': 'na', 'n': 'na'}",60,na,na,650,-3.37474,76.23,76.23,54.0402,na,na,na,na,0.0,na,na,na,,na,na,na,na,na,na,na,na,na,na,na,na,na,na,na,0.0423,na,3D-bulk,[],<a href=https://www.ctcms.nist.gov/~knc6/stati...,bulk,na,129,tetragonal,5.956,na,[],8,na,na,na,na,mp-1080455,-As-Cu-Si-Ti
1,JVASP-86097,221,Pm-3m,DyB6,-0.41596,OptB88vdW,0.000,"{'lattice_mat': [[4.089078911208881, 0.0, 0.0]...",na,0.0,na,na,"{'p': 'na', 'n': 'na'}",65,na,na,1000,-5.79186,140.9742,140.9742,140.9742,na,na,na,na,0.0,na,na,na,"613601,613595,150623",na,na,na,na,na,na,na,na,na,na,na,na,na,na,na,0.0456,na,3D-bulk,[],<a href=https://www.ctcms.nist.gov/~knc6/stati...,bulk,na,221,cubic,5.522,na,"[OPT-LOPTICS,JVASP-86097.zip,https://ndownload...",7,na,na,na,na,mp-568319,-B-Dy
2,JVASP-64906,119,I-4m2,Be2OsRu,0.04847,OptB88vdW,0.000,"{'lattice_mat': [[-1.833590720595598, 1.833590...",na,0.0,na,na,"{'p': 'na', 'n': 'na'}",60,na,na,1100,-4.65924,215.4488,215.4488,159.4368,na,na,na,na,0.0,23.94,na,na,,na,na,na,na,na,na,na,na,na,na,na,na,na,na,na,0.3183,na,intercalated ion,"[[Be, a, -1.504, 0.001, 0.001, 0.001, -1.504, ...",<a href=https://www.ctcms.nist.gov/~knc6/stati...,bulk,na,119,tetragonal,10.960,na,"[OPT-LOPTICS,JVASP-64906.zip,https://ndownload...",4,na,na,na,na,auid-3eaf68dd483bf4f4,-Be-Os-Ru
3,JVASP-98225,14,P2_1/c,KBi,-0.44140,OptB88vdW,0.472,"{'lattice_mat': [[7.2963518353359165, 0.0, 0.0...",na,0.0,na,na,"{'p': 'na', 'n': 'na'}",40,na,na,500,-0.42496,na,na,na,na,na,na,na,0.0,na,na,na,55065,na,na,na,na,na,na,na,na,na,na,na,na,na,na,na,0.0000,na,intercalated ion,[],<a href=https://www.ctcms.nist.gov/~knc6/stati...,bulk,na,14,monoclinic,5.145,na,[],32,na,na,na,na,mp-31104,-Bi-K
4,JVASP-10,164,P-3m1,VSe2,-0.71026,OptB88vdW,0.000,"{'lattice_mat': [[1.6777483798834445, -2.90594...",na,0.0,na,"[[136.4, 27.8, 17.5, 0.0, -5.5, 0.0], [27.8, 1...","{'p': [0.0, 0.0, 0.0], 'n': [0.0, 0.0, 0.0]}",40,na,na,500,-3.87823,47.029,47.029,28.1625,79.071,78.7247,18.006,"[-0.22, -0.1, -0.1, 142.22, 142.22, 192.04, 19...",0.0,89.678,0.0,0.0,"86520,652160,652163,652158",0.0,0.0,54.202007,44.924312,9.277695,342.84,204.14,18.273333,17.486667,657.17,614.083333,2033190.976667,2045950.983333,2798132623333333.0,2811808060000000.0,0.0156,na,2D-bulk,"[[V, a, 22.568, 0.0, 0.0, 0.0, 22.559, 0.0, 0....",<a href=https://www.ctcms.nist.gov/~knc6/stati...,bulk,83.78,164,trigonal,5.718,0.23,"[FD-ELAST,JVASP-10.zip,https://ndownloader.fig...",3,48.79,33.05,0.0,na,mp-694,-Se-V


In [12]:
dft_3d = data('dft_3d')
dft_2d = data('dft_2d')

Obtaining 3D dataset 76k ...
Reference:https://www.nature.com/articles/s41524-020-00440-1
Other versions:https://doi.org/10.6084/m9.figshare.6815699
Loading the zipfile...
Loading completed.
Obtaining 2D dataset 1.1k ...
Reference:https://www.nature.com/articles/s41524-020-00440-1
Other versions:https://doi.org/10.6084/m9.figshare.6815705


100%|██████████| 8.39M/8.39M [00:00<00:00, 10.7MiB/s]


Loading the zipfile...
Loading completed.


In [14]:
print(len(dft_3d),len(dft_2d))

75993 1103


In [15]:
#import pandas as pd
df = pd.DataFrame(dft_3d)
df.head()

,jid,spg_number,spg_symbol,formula,formation_energy_peratom,func,optb88vdw_bandgap,atoms,slme,magmom_oszicar,spillage,elastic_tensor,effective_masses_300K,kpoint_length_unit,maxdiff_mesh,maxdiff_bz,encut,optb88vdw_total_energy,epsx,epsy,epsz,mepsx,mepsy,mepsz,modes,magmom_outcar,max_efg,avg_elec_mass,avg_hole_mass,icsd,dfpt_piezo_max_eij,dfpt_piezo_max_dij,dfpt_piezo_max_dielectric,dfpt_piezo_max_dielectric_electronic,dfpt_piezo_max_dielectric_ionic,max_ir_mode,min_ir_mode,n-Seebeck,p-Seebeck,n-powerfact,p-powerfact,ncond,pcond,nkappa,pkappa,ehull,Tc_supercon,dimensionality,efg,xml_data_link,typ,exfoliation_energy,spg,crys,density,poisson,raw_files,nat,bulk_modulus_kv,shear_modulus_gv,mbj_bandgap,hse_gap,reference,search
0,JVASP-90856,129,P4/nmm,TiCuSiAs,-0.42762,OptB88vdW,0.000,"{'lattice_mat': [[3.566933224304235, 0.0, -0.0...",na,0.0,na,na,"{'p': 'na', 'n': 'na'}",60,na,na,650,-3.37474,76.23,76.23,54.0402,na,na,na,na,0.0,na,na,na,,na,na,na,na,na,na,na,na,na,na,na,na,na,na,na,0.0423,na,3D-bulk,[],<a href=https://www.ctcms.nist.gov/~knc6/stati...,bulk,na,129,tetragonal,5.956,na,[],8,na,na,na,na,mp-1080455,-As-Cu-Si-Ti
1,JVASP-86097,221,Pm-3m,DyB6,-0.41596,OptB88vdW,0.000,"{'lattice_mat': [[4.089078911208881, 0.0, 0.0]...",na,0.0,na,na,"{'p': 'na', 'n': 'na'}",65,na,na,1000,-5.79186,140.9742,140.9742,140.9742,na,na,na,na,0.0,na,na,na,"613601,613595,150623",na,na,na,na,na,na,na,na,na,na,na,na,na,na,na,0.0456,na,3D-bulk,[],<a href=https://www.ctcms.nist.gov/~knc6/stati...,bulk,na,221,cubic,5.522,na,"[OPT-LOPTICS,JVASP-86097.zip,https://ndownload...",7,na,na,na,na,mp-568319,-B-Dy
2,JVASP-64906,119,I-4m2,Be2OsRu,0.04847,OptB88vdW,0.000,"{'lattice_mat': [[-1.833590720595598, 1.833590...",na,0.0,na,na,"{'p': 'na', 'n': 'na'}",60,na,na,1100,-4.65924,215.4488,215.4488,159.4368,na,na,na,na,0.0,23.94,na,na,,na,na,na,na,na,na,na,na,na,na,na,na,na,na,na,0.3183,na,intercalated ion,"[[Be, a, -1.504, 0.001, 0.001, 0.001, -1.504, ...",<a href=https://www.ctcms.nist.gov/~knc6/stati...,bulk,na,119,tetragonal,10.960,na,"[OPT-LOPTICS,JVASP-64906.zip,https://ndownload...",4,na,na,na,na,auid-3eaf68dd483bf4f4,-Be-Os-Ru
3,JVASP-98225,14,P2_1/c,KBi,-0.44140,OptB88vdW,0.472,"{'lattice_mat': [[7.2963518353359165, 0.0, 0.0...",na,0.0,na,na,"{'p': 'na', 'n': 'na'}",40,na,na,500,-0.42496,na,na,na,na,na,na,na,0.0,na,na,na,55065,na,na,na,na,na,na,na,na,na,na,na,na,na,na,na,0.0000,na,intercalated ion,[],<a href=https://www.ctcms.nist.gov/~knc6/stati...,bulk,na,14,monoclinic,5.145,na,[],32,na,na,na,na,mp-31104,-Bi-K
4,JVASP-10,164,P-3m1,VSe2,-0.71026,OptB88vdW,0.000,"{'lattice_mat': [[1.6777483798834445, -2.90594...",na,0.0,na,"[[136.4, 27.8, 17.5, 0.0, -5.5, 0.0], [27.8, 1...","{'p': [0.0, 0.0, 0.0], 'n': [0.0, 0.0, 0.0]}",40,na,na,500,-3.87823,47.029,47.029,28.1625,79.071,78.7247,18.006,"[-0.22, -0.1, -0.1, 142.22, 142.22, 192.04, 19...",0.0,89.678,0.0,0.0,"86520,652160,652163,652158",0.0,0.0,54.202007,44.924312,9.277695,342.84,204.14,18.273333,17.486667,657.17,614.083333,2033190.976667,2045950.983333,2798132623333333.0,2811808060000000.0,0.0156,na,2D-bulk,"[[V, a, 22.568, 0.0, 0.0, 0.0, 22.559, 0.0, 0....",<a href=https://www.ctcms.nist.gov/~knc6/stati...,bulk,83.78,164,trigonal,5.718,0.23,"[FD-ELAST,JVASP-10.zip,https://ndownloader.fig...",3,48.79,33.05,0.0,na,mp-694,-Se-V


# <font color = green> Filtering dataset based on a set of elements

In [17]:
from jarvis.core.atoms import Atoms
def req_elements(atoms_dict = {}, my_element = ['Li','Co','Mn','O']):
  atoms = Atoms.from_dict(atoms_dict)
  elements = atoms.elements
  if set(my_element) == set(elements):
    return True
  else:
    return np.nan

df['el'] = df['atoms'].apply(lambda x: req_elements(atoms_dict = x, my_element = ['Li','Co','Mn','O']))
df1 = df.dropna()
# Exercise 1: Check if you get similar number on https://jarvis.nist.gov/jarvisdft/
# Exercise 2: Filter for ['Bi','Te'] instead with spacegroup 166(R-3m)
print ('#entries:',len(df1))
df1

#entries: 392


,jid,spg_number,spg_symbol,formula,formation_energy_peratom,func,optb88vdw_bandgap,atoms,slme,magmom_oszicar,spillage,elastic_tensor,effective_masses_300K,kpoint_length_unit,maxdiff_mesh,maxdiff_bz,encut,optb88vdw_total_energy,epsx,epsy,epsz,mepsx,mepsy,mepsz,modes,magmom_outcar,max_efg,avg_elec_mass,avg_hole_mass,icsd,dfpt_piezo_max_eij,dfpt_piezo_max_dij,dfpt_piezo_max_dielectric,dfpt_piezo_max_dielectric_electronic,dfpt_piezo_max_dielectric_ionic,max_ir_mode,min_ir_mode,n-Seebeck,p-Seebeck,n-powerfact,p-powerfact,ncond,pcond,nkappa,pkappa,ehull,Tc_supercon,dimensionality,efg,xml_data_link,typ,exfoliation_energy,spg,crys,density,poisson,raw_files,nat,bulk_modulus_kv,shear_modulus_gv,mbj_bandgap,hse_gap,reference,search,el
8460,JVASP-43812,12,C2/m,LiMnCo3O8,-1.40133,OptB88vdW,0.000,"{'lattice_mat': [[5.727193929077369, -0.056329...",na,1.053,na,na,"{'p': [0.0, 0.0, 0.0], 'n': [0.0, 0.0, 0.0]}",30,na,na,1300,-4.88948,42.0615,117.1264,78.5993,na,na,na,na,-1.041,na,0.0,0.0,,na,na,na,na,na,na,na,45.676667,44.653333,1040.143333,1011.51,299105.2,303437.36,417741249999999.75,422221076666666.5625,0.0725,na,intercalated ion,[],<a href=https://www.ctcms.nist.gov/~knc6/stati...,bulk,na,12,monoclinic,4.632,na,"[OPT-LOPTICS,JVASP-43812.zip,https://ndownload...",13,na,na,0.0,na,mp-777603,-Co-Li-Mn-O,True
8675,JVASP-43333,166,R-3m,Li2Mn3CoO8,-1.91342,OptB88vdW,0.000,"{'lattice_mat': [[0.010825708305805, 4.0575690...",na,8.0,na,na,"{'p': [0.0, 0.0, 0.0], 'n': [0.0, 0.0, 0.0]}",30,na,na,1300,-5.39948,94.3008,94.3008,94.3008,na,na,na,na,7.811,na,0.0,0.0,,na,na,na,na,na,na,na,-24.373333,-25.523333,186.503333,195.183333,323993.616667,322499.25,405472649999999.9375,399677370000000.0,0.0086,na,intercalated ion,[],<a href=https://www.ctcms.nist.gov/~knc6/stati...,bulk,na,166,trigonal,4.562,na,"[OPT-LOPTICS,JVASP-43333.zip,https://ndownload...",14,na,na,na,na,mp-775651,-Co-Li-Mn-O,True
8712,JVASP-43813,12,C2/m,Li2MnCo3O8,-1.57553,OptB88vdW,0.000,"{'lattice_mat': [[5.701829646182634, -7.580242...",na,2.131,na,na,"{'p': [0.0, 0.0, 0.0], 'n': [0.0, 0.0, 0.0]}",30,na,na,1300,-4.88055,27.7454,84.8888,49.3937,195.8023,169.2996,266.9146,na,2.085,na,0.0,0.0,,na,na,na,na,na,na,na,56.75,60.373333,760.103333,838.916667,109873.81,111879.26,287389103333333.375,296378680000000.1875,0.0712,na,intercalated ion,[],<a href=https://www.ctcms.nist.gov/~knc6/stati...,bulk,na,12,monoclinic,4.707,na,"[OPT-LOPTICS,JVASP-43813.zip,https://ndownload...",14,na,na,0.0,na,mp-777604,-Co-Li-Mn-O,True
14411,JVASP-43110,160,R3m,Li2Mn3CoO8,-1.86147,OptB88vdW,0.066,"{'lattice_mat': [[5.772029724729537, 0.0163626...",na,5.813,na,na,"{'p': [0.0, 0.0, 0.0], 'n': [0.0, 0.0, 0.0]}",30,na,na,1300,-5.34752,132.7134,174.1565,184.5174,na,na,na,na,5.704,na,0.0,0.0,,na,na,na,na,na,na,na,-100.36,-84.376667,1190.806667,961.163333,51854.113333,42839.396667,176721440000000.0,165015723333333.3125,0.0605,na,intercalated ion,[],<a href=https://www.ctcms.nist.gov/~knc6/stati...,bulk,na,160,trigonal,4.498,na,"[OPT-LOPTICS,JVASP-43110.zip,https://ndownload...",14,na,na,na,na,mp-775027,-Co-Li-Mn-O,True
18407,JVASP-101528,12,C2/m,Li2MnCoO4,-1.85890,OptB88vdW,0.000,"{'lattice_mat': [[2.7492060708413195, 0.047863...",na,2.001,na,na,"{'p': 'na', 'n': 'na'}",30,na,na,500,-4.94555,na,na,na,na,na,na,na,1.961,na,na,na,,na,na,na,na,na,na,na,na,na,na,na,na,na,na,na,0.0453,na,intercalated ion,[],<a href=https://www.ctcms.nist.gov/~knc6/stati...,bulk,na,12,monoclinic,4.804,na,[],8,na,na,na,na,mp-1173902,-Co-Li-Mn-O,True
19358,JVASP-105589,10,P2/m,Li2MnCoO4,-1.78200,OptB88vdW,0.000,"{'lattice_mat': [[2.837286857409075, -0.000482...",na,0.063,na,na,"{'p': 'na', 'n': 'na'}",30,na,na,500,-4.86864,na,na,na,na,na,na,na,0.063,na,na,na,,na,na,na,na,na,na,na,na,na,na,na,na,na,na,na,0.1222,na,intercalated ion,[],<a href=https://www.ctcms.nist.gov/~knc6/stati...,bulk,na,10,monoclinic,4.840,na,[],8,na,na,na,na,mp-753482,-Co-Li-Mn-O,True
19485,JVASP-43152,64,Cmce,Li2Mn2CoO6,-1.79537,OptB88vdW,0.000,"{'lattice_mat': [[4.9077485588

In [23]:
df1.describe()

,formation_energy_peratom,optb88vdw_bandgap,optb88vdw_total_energy,ehull,density,nat
count,392.000000,392.000000,392.000000,392.000000,392.000000,392.000000
mean,-1.881186,0.130638,-4.848118,0.057724,4.477747,22.466837
std,0.098091,0.205996,0.135678,0.043254,0.184154,4.900433
min,-2.065180,0.000000,-5.399480,0.000000,3.999000,8.000000
25%,-1.927490,0.000000,-4.926210,0.020550,4.350750,20.000000
50%,-1.890780,0.000000,-4.794755,0.049900,4.464000,24.000000
75%,-1.840135,0.282500,-4.757948,0.080950,4.538000,28.000000
max,-1.338160,0.858000,-4.583240,0.250700,5.236000,30.000000


In [26]:
#print(df1.isna().value_counts)

# <font color =green> Filtering for 'Li','Co','Mn','O'

In [27]:
numerical_col = list(df.describe().columns)
col_categorical = list(set(df.columns).difference(numerical_col))
remove_list = ['effective_masses_300K','reference','raw_files']

col_numerical = [i for i in numerical_col if i not in remove_list]

In [28]:
categorical_col = []
for m, n in df.items():
  if n.dtype == 'object':
    categorical_col.append(m)

print(categorical_col,len(categorical_col))



['jid', 'spg_number', 'spg_symbol', 'formula', 'func', 'atoms', 'slme', 'magmom_oszicar', 'spillage', 'elastic_tensor', 'effective_masses_300K', 'kpoint_length_unit', 'maxdiff_mesh', 'maxdiff_bz', 'encut', 'epsx', 'epsy', 'epsz', 'mepsx', 'mepsy', 'mepsz', 'modes', 'magmom_outcar', 'max_efg', 'avg_elec_mass', 'avg_hole_mass', 'icsd', 'dfpt_piezo_max_eij', 'dfpt_piezo_max_dij', 'dfpt_piezo_max_dielectric', 'dfpt_piezo_max_dielectric_electronic', 'dfpt_piezo_max_dielectric_ionic', 'max_ir_mode', 'min_ir_mode', 'n-Seebeck', 'p-Seebeck', 'n-powerfact', 'p-powerfact', 'ncond', 'pcond', 'nkappa', 'pkappa', 'Tc_supercon', 'dimensionality', 'efg', 'xml_data_link', 'typ', 'exfoliation_energy', 'spg', 'crys', 'poisson', 'raw_files', 'bulk_modulus_kv', 'shear_modulus_gv', 'mbj_bandgap', 'hse_gap', 'reference', 'search', 'el'] 59


# <font color = green> Count number of entries for each property

In [29]:
for i in df.columns.values:
  val=df[i].replace('na',np.nan).dropna().values
  print(i,len(val))

jid 75993
spg_number 75993
spg_symbol 75993
formula 75993
formation_energy_peratom 75993
func 75993
optb88vdw_bandgap 75993
atoms 75993
slme 9770


/tmp/ipython-input-948664666.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  val=df[i].replace('na',np.nan).dropna().values


magmom_oszicar 71320
spillage 11377
elastic_tensor 25513
effective_masses_300K 75993
kpoint_length_unit 75671
maxdiff_mesh 5861
maxdiff_bz 5861
encut 75670
optb88vdw_total_energy 75993
epsx 52168
epsy 52168
epsz 52168
mepsx 18293
mepsy 18293
mepsz 18293
modes 13910
magmom_outcar 74261
max_efg 11871
avg_elec_mass 17645
avg_hole_mass 17645
icsd 75993
dfpt_piezo_max_eij 4799
dfpt_piezo_max_dij 3347
dfpt_piezo_max_dielectric 4706
dfpt_piezo_max_dielectric_electronic 4809
dfpt_piezo_max_dielectric_ionic 4809
max_ir_mode 4805
min_ir_mode 4809
n-Seebeck 23218
p-Seebeck 23218
n-powerfact 23218
p-powerfact 23218
ncond 23218
pcond 23218
nkappa 23218
pkappa 23218
ehull 75993
Tc_supercon 1058
dimensionality 75560
efg 75993
xml_data_link 75993
typ 75993
exfoliation_energy 813
spg 75993
crys 75993
density 75993
poisson 23597
raw_files 75993
nat 75993
bulk_modulus_kv 23824
shear_modulus_gv 23824
mbj_bandgap 19805
hse_gap 56
reference 75993
search 75993
el 392


In [30]:
col_numerical

['formation_energy_peratom',
 'optb88vdw_bandgap',
 'optb88vdw_total_energy',
 'ehull',
 'density',
 'nat']